In [ ]:
!pip install keras-resnet==0.1.0

In [1]:
import numpy as np
import tensorflow as tf
seed = 1
np.random.seed(seed)
tf.set_random_seed(seed)

import keras
import tempfile
import keras.models

from keras import backend as K 
from shift_detector import *
from shift_locator import *
from shift_applicator import *
from data_utils import *
from shared_utils import *
import os
import sys
import scipy.io

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc

/Users/mahzadkhoshlessan/miniconda3/envs/FSDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/mahzadkhoshlessan/miniconda3/envs/FSDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/mahzadkhoshlessan/miniconda3/envs/FSDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/mahzad

In [2]:
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
rc('axes', labelsize=22)
rc('xtick', labelsize=22)
rc('ytick', labelsize=22)
rc('legend', fontsize=13)

mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']


def clamp(val, minimum=0, maximum=255):
    if val < minimum:
        return minimum
    if val > maximum:
        return maximum
    return val


def colorscale(hexstr, scalefactor):
    hexstr = hexstr.strip('#')

    if scalefactor < 0 or len(hexstr) != 6:
        return hexstr

    r, g, b = int(hexstr[:2], 16), int(hexstr[2:4], 16), int(hexstr[4:], 16)

    r = clamp(r * scalefactor)
    g = clamp(g * scalefactor)
    b = clamp(b * scalefactor)

    return "#%02x%02x%02x" % (int(r), int(g), int(b))


def errorfill(x, y, yerr, color=None, alpha_fill=0.2, ax=None, fmt='-o', label=None):
    ax = ax if ax is not None else plt.gca()
    if color is None:
        color = next(ax._get_lines.prop_cycler)['color']
    if np.isscalar(yerr) or len(yerr) == len(y):
        ymin = y - yerr
        ymax = y + yerr
    elif len(yerr) == 2:
        ymin, ymax = yerr
    ax.semilogx(x, y, fmt, color=color, label=label)
    ax.fill_between(x, np.clip(ymax, 0, 1), np.clip(ymin, 0, 1), color=color, alpha=alpha_fill)


def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            keras.models.save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = { 'model_str': model_str }
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = keras.models.load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = keras.models.Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__


linestyles = ['-', '-.', '--', ':']
brightness = [1.25, 1.0, 0.75, 0.5]
format = ['-o', '-h', '-p', '-s', '-D', '-<', '->', '-X']
markers = ['o', 'h', 'p', 's', 'D', '<', '>', 'X']
colors_old = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']
colors = ['#2196f3', '#f44336', '#9c27b0', '#64dd17', '#009688', '#ff9800', '#795548', '#607d8b']

/Users/mahzadkhoshlessan/miniconda3/envs/FSDL/lib/python3.6/site-packages/ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  


In [3]:
make_keras_picklable()

In [4]:
datset = 'mnist'
test_type = 'univ'

In [5]:
path = './paper_results/'
path += test_type + '/'
path += datset + '_'
path += 'adversarial_shift' + '/'
if not os.path.exists(path):
    os.makedirs(path)

In [6]:
dr_techniques = [DimensionalityReduction.NoRed.value, DimensionalityReduction.PCA.value, 
                 DimensionalityReduction.UAE.value, DimensionalityReduction.TAE.value] 
# dr_techniques = [DimensionalityReduction.NoRed.value, DimensionalityReduction.PCA.value, DimensionalityReduction.SRP.value, 
#                  DimensionalityReduction.UAE.value, DimensionalityReduction.TAE.value, DimensionalityReduction.BBSDs.value, 
#                  DimensionalityReduction.BBSDh.value] 

In [7]:
dr_techniques_plot = dr_techniques.copy()
dr_techniques_plot.append(DimensionalityReduction.Classif.value)

In [8]:
test_types = [td.value for td in TestDimensionality]

In [9]:
test_types

[1, 2, 3]

In [10]:
od_tests = [OnedimensionalTest.KS.value]
md_tests = []
samples = [10 , 20, 50, 100, 200, 500, 1000, 10000]

In [11]:
difference_samples = 20

In [12]:
# Number of random runs to average results over.
random_runs = 1

# Significance level.
sign_level = 0.05

# Whether to calculate accuracy for malignancy quantification.
calc_acc = True

In [13]:
shifts = ['adversarial_shift_0.1',
          'adversarial_shift_0.5',
          'adversarial_shift_1.0']

In [14]:
# Stores p-values for all experiments of a shift class.
samples_shifts_rands_dr_tech = np.ones((len(samples), len(shifts), random_runs, len(dr_techniques_plot))) * (-1)

In [15]:
red_dim = -1
red_models = [None] * len(DimensionalityReduction)

In [16]:
path

'./paper_results/univ/mnist_adversarial_shift/'

In [17]:
for shift_idx, shift in enumerate(shifts):
    shift_path = path + shift + '/'
    if not os.path.exists(shift_path):
        os.makedirs(shift_path)
        
    # Stores p-values for a single shift.
    rand_run_p_vals = np.ones((len(samples), len(dr_techniques_plot), random_runs)) * (-1)
    
    # Stores accuracy values for malignancy detection.
    val_accs = np.ones((random_runs, len(samples))) * (-1)
    te_accs = np.ones((random_runs, len(samples))) * (-1)
    dcl_accs = np.ones((len(samples), random_runs)) * (-1)
    
    # Average over a few random runs to quantify robustness.
    for rand_run in range(random_runs):
        print("Random run %s" % rand_run)

        rand_run_path = shift_path + str(rand_run) + '/'
        if not os.path.exists(rand_run_path):
            os.makedirs(rand_run_path)

        np.random.seed(rand_run)
        tf.set_random_seed(rand_run)

        # Load data.
        (X_tr_orig, y_tr_orig), (X_val_orig, y_val_orig), (X_te_orig, y_te_orig), orig_dims, nb_classes = \
            import_dataset(datset, shuffle=True)
        
        print(orig_dims)
        X_tr_orig = normalize_datapoints(X_tr_orig, 255.)
        X_te_orig = normalize_datapoints(X_te_orig, 255.)
        X_val_orig = normalize_datapoints(X_val_orig, 255.)
        
        (X_te_1, y_te_1) = apply_shift(X_te_orig, y_te_orig, shift, orig_dims, datset)   
        
        X_te_2 , y_te_2 = random_shuffle(X_te_1, y_te_1)
        
        # Check detection performance for different numbers of samples from test.
        for si, sample in enumerate(samples):

            print("Sample %s" % sample)

            sample_path = rand_run_path + str(sample) + '/'
            if not os.path.exists(sample_path):
                os.makedirs(sample_path)

            X_te_3 = X_te_2[:sample,:]
            y_te_3 = y_te_2[:sample]
            
            X_val_3 = X_val_orig[:sample,:]
            y_val_3 = y_val_orig[:sample]
            
            X_tr_3 = np.copy(X_tr_orig)
            y_tr_3 = np.copy(y_tr_orig)
            
            # Detect shift.
            shift_detector = ShiftDetector(dr_techniques, test_types, od_tests, md_tests, sign_level, red_models,
                                           sample, datset)
#             (od_decs, ind_od_decs, ind_od_p_vals), \
#             (md_decs, ind_md_decs, ind_md_p_vals), \
            out_val_red, out_te_red, red_dim, red_models, val_acc, te_acc = shift_detector.detect_data_shift(X_tr_3, y_tr_3, X_val_3, y_val_3,
                                                                                    X_te_3, y_te_3, orig_dims,
                                                                                    nb_classes)
            
            
            
            out_val_red = {key:normalize_datapoints(out_val_red[key], 1./255.) for key in out_val_red.keys()}
            out_te_red = {key:normalize_datapoints(out_te_red[key], 1./255.) for key in out_te_red.keys()}
            val_file = 'val_mat_shift1_{}_{}_{}.mat'.format(shift,sample,rand_run)
            test_file = 'test_mat_shift1_{}_{}_{}.mat'.format(shift,sample,rand_run)
            
            scipy.io.savemat(val_file, mdict=out_val_red, oned_as='row')
            scipy.io.savemat(test_file, mdict=out_te_red, oned_as='row')
            
            
#             print("Found pretrained model, loading...")
#             ckpt = torch.load(pretrained_filename)
#             flow.load_state_dict(ckpt['state_dict'])
#             result = ckpt.get("result", None)
            
#             logp = flow._get_likelihood(sample_imgs, return_ll=True)
#             logp = logp.cpu().data.numpy()

Random run 0
(28, 28, 1)
(28, 28, 1)
Small adversarial Shift
Sample 10
NoRed
None
(10, 784)
(10, 28, 28)
PCA
PCA(n_components=32)
(10, 32)
(10, 28, 28) (10, 28, 28)
UAE
Hello There
(10, 28, 28, 1)
(10, 28, 28)
TAE
Hello There
Train on 50000 samples, validate on 10 samples
Epoch 1/200
50000/50000 [==============================] - 65s 1ms/step - loss: 0.4053 - val_loss: 0.4168
Epoch 2/200
50000/50000 [==============================] - 64s 1ms/step - loss: 0.3861 - val_loss: 0.4052
Epoch 3/200
50000/50000 [==============================] - 81s 2ms/step - loss: 0.3741 - val_loss: 0.3933
Epoch 4/200
50000/50000 [==============================] - 80s 2ms/step - loss: 0.2741 - val_loss: 0.2397
Epoch 5/200
50000/50000 [==============================] - 70s 1ms/step - loss: 0.2030 - val_loss: 0.2135
Epoch 6/200
50000/50000 [==============================] - 68s 1ms/step - loss: 0.1901 - val_loss: 0.2012
Epoch 7/200
50000/50000 [==============================] - 64s 1ms/step - loss: 0.1828 - va

In [ ]:
scipy.io.loadmat(val_file)

In [ ]:
np.shape(a)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
img = cv2.imread('img.png') #you can use any image you want.
plt.imshow(img)

In [ ]:
np.shape(img)

In [ ]:
blue,green,red = cv2.split(img) 

In [ ]:
np.shape(blue)

In [ ]:
pca = PCA(20)

In [ ]:
red_transformed = pca.fit_transform(red)

In [ ]:
red_inverted = pca.inverse_transform(red_transformed)

In [ ]:
green_transformed = pca.fit_transform(green)
green_inverted = pca.inverse_transform(green_transformed)
 
#Applying to Blue channel and then applying inverse transform to transformed array.
blue_transformed = pca.fit_transform(blue)
blue_inverted = pca.inverse_transform(blue_transformed)

In [ ]:
pca.inverse_transform?

In [ ]:
np.shape(red_inverted)

In [ ]:
np.shape(red_transformed)

In [ ]:
img_compressed = (np.dstack((red_inverted, red_inverted, red_inverted))).astype(np.uint8)

In [ ]:
plt.imshow(img_compressed)